In [ ]:
# the scipy version packaged with colab is not tolerant of misformated WAV files.
# install the latest version.
!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install torchaudio
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install


In [ ]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HuggingFace hub.
tts = TextToSpeech()

In [ ]:
# Import necessary libraries
import os
from google.colab import files

# Upload EPUB file 
uploaded = files.upload()

# Create a directory to store converted EPUB files
if not os.path.exists("converted_epub"):
    os.makedirs("converted_epub")

# Loop through the uploaded files
for epub_file_name in uploaded.keys():
    # Convert EPUB to TXT using ebook-convert from Calibre
    txt_file_name = os.path.splitext(epub_file_name)[0] + ".txt"
    !ebook-convert "{epub_file_name}" "converted_epub/{txt_file_name}"

    # Print conversion success message
    print(f"Successfully converted {epub_file_name} to {txt_file_name} and stored in 'converted_epub' folder.")

In [ ]:
# upload an epub file to convert to text file
from google.colab import files
uploaded = files.upload()

# convert epub to text file and save to folder named converted_epub
!mkdir converted_epub
!ebook-convert *.epub converted_epub/converted.txt


In [ ]:
# Select converted file to use 
import os
import glob
import pandas as pd

path = 'converted_epub'
all_files = glob.glob(os.path.join(path, "*.txt"))
all_files.sort()
all_files

# Select file to use
file = all_files[0]
file


In [ ]:
# convert selected file to speech
# this will take a while
# the output will be saved to the folder named output
!mkdir output
tts.convert(file, 'output/output.wav')

# download output file
from google.colab import files
files.download('output/output.wav')

# play output file
IPython.display.Audio('output/output.wav')



In [ ]:
# This is the text that will be spoken.
text = "Thanks for reading this article. I hope you learned something."



# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "fast"

In [ ]:
# Optionally, upload use your own voice by running the next two cells. I recommend
# you upload at least 2 audio clips. They must be a WAV file, 6-10 seconds long.
CUSTOM_VOICE_NAME = "martin"

import os
from google.colab import files

custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder)
for i, file_data in enumerate(files.upload().values()):
  with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
    f.write(file_data)

In [ ]:
# Generate speech with the custotm voice.
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')